# Using NeuNetS Generated Model to Locally Perform Inference on Sample Images

This notebook imports a **Keras** (TensorFlow) model which was designed and trained using *NeuNetS*. *NeuNetS* designs and trains a custom neural network, which users may then download. This download produces the file neunets_output.tar.gz. This compressed file contains two objects, keras_model.hdf5, and metadata.json. 

The metadata file contains information necessary for pre- and post-processing input images to be compatible with the **Keras** model. Since **Keras** requires that classes have integer labels, this file contains the mapping necessary to find the actual label corresponding to the predicted class. 

The user can input **raw .jpg or .png images** and use this model to infer their class. 

## 1. Install NeunetsProcessor

Here we install the library required for inferencing on the model.

In [ ]:
!pip install --upgrade git+https://github.com/pmservice/NeuNetS/#egg=neunets_processor\&subdirectory=neunets_processor

**Tip**: Restart the kernel (**Kernel** -> **Restart**)

## 2. Unpack the Model and Metadata

This is the location of the model. A .tar.gz file is downloaded from *NeuNetS*, which contains the model
along with a .json file which specifies the mapping between class label names and 
the class indices used by **Keras**. 

In [ ]:
#This must be the absolute path.
#The file neunets_output.tar.gz should have been downloaded to this directory from the NeuNetS service. 
model_location = '/path_to_model/'

In [ ]:
# Install os library to handle file operations
import os

Go to the directory containing the compressed model file and unpack it. Then return to the current directory. 

In [ ]:
#change to model directory and unpack
#If the model file has been renamed, this must be changed in this cell. 
cwd = os.getcwd()
os.chdir(model_location)
assert os.path.isfile("neunets_output.tar.gz")
!gunzip neunets_output.tar.gz
!tar -xvf neunets_output.tar
os.chdir(cwd)

Names of the files containing the model and the label mapping. These will be used to initialize an image processor instance which can then perform inference on images input by the user. 

In [ ]:
model_file = os.path.join(model_location,"keras_model.hdf5")
metadata_file = os.path.join(model_location,"metadata.json")

In [ ]:
from neunets_processor.image import image_processor

In [ ]:
#Initialize image processor from model and metadata file
processor = image_processor.ImageProcessor(model_file, metadata_file)

## 3. Classify an Image

Importing images into the notebook requires the Python Imaging Library

In [ ]:
# Python Image Processing Library
from PIL import Image

The user supplies an image here in .png or .jpg format.

In [ ]:
#open test image
test_image = Image.open('/path_to_image/test-image.jpg')
display(test_image)

Using the processor.predict method provides the class label predicted by the model. 

In [ ]:
#Find the predicted class
predicted_class = processor.predict([test_image])
print("predicted class label is: ", predicted_class)

This is all that is necessary to use the *NeuNetS* synthesized model to classify images input by the user. For those who want more fine-grained control over the inferencing process, a more detailed description is given below. 

## 4. Details of Processing

In order to obtain a prediction, the method used above is sufficient. If the user wishes to obtain more granular output during this process, they can invoke the subroutines of this method. 

The processor.predict method wraps the three steps below:
1. Preprocess raw input image into correct input format for the **Keras** model. 
2. Use the model loaded by the processor to make a prediction. 
3. Extract prediction from **Keras** model.predict output. 

These intermediate steps are documented below. 

### 4.1 Preprocess the Image
In order to format a raw jpg or png image to be input into the *NeuNetS*/**Keras** model, it must be preprocessed. This step resizes the image to fit the dimensions of the model's input layer, and normalizes the image to obtain an accurate prediction. 

In [ ]:
preprocessed_image = processor.preprocess([test_image])

### 4.2 Call model.predict() from **Keras**
The image_processor library uses the **Keras** framework to make a prediction with the model. The user can also do this step on their own, without utilizing the library. 

First import the model class and load_model function from **Keras**. (TensorFlow and **Keras** will have been installed along with text_processor.) Then call model.predict() to get an array of probabilities. 

In [ ]:
from keras.models import Model
from keras.models import load_model

model = load_model(model_file)
probabilities = model.predict(preprocessed_image)
probabilities

### 4.3 Postprocess to Obtain Class Labels
The postprocess method in the image_processor uses the metadata file that was packaged with the *NeuNetS* model to convert this array of probabilities into a dictionary containing the class labels as keys and the probabilities assigned to those classes as values.

In [ ]:
result = processor.postprocess(probabilities[0])
result

### Author: *NeuNetS* Team

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.